In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import random
from shapely.geometry import Point
from shapely import contains_xy

# Import functions from functions.py
from functions import generate_random_location_within_map_domain, get_flood_depth, calculate_basic_flood_damage, floodplain_multipolygon
import networkx as nx
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from mesa import batch_run
import geopandas as gpd
import rasterio as rs
import matplotlib.pyplot as plt
# Import the agent class(es) from agents.py
from agents import Households
from agents import Government
from rbb import OrganizationInstrument

# Import functions from functions.py
from functions import get_flood_map_data, calculate_basic_flood_damage
from functions import map_domain_gdf, floodplain_gdf

from model import AdaptationModel

In [2]:
# # Initialize the Adaptation Model with 50 household agents.
# model = AdaptationModel(
#     seed = random.seed(42),
#     number_of_households=50, 
#     flood_map_choice="harvey", 
#     network="watts_strogatz",
#     government_decision = False) # flood_map_choice can be "harvey", "100yr", or "500yr"

# for step in range(120):
#     model.step()

# model_data = model.datacollector.get_model_vars_dataframe()
# # model_data
# # fig, (ax1, ax2) = plt.subplot(2, 1)
# sns.lineplot(model_data["total_adapted_households"], label='cumulatief')
# sns.lineplot(model_data['total_decisions_to_adapt'], label='per stap')
# plt.legend()
# agent_data = model.datacollector.get_agent_vars_dataframe()

In [3]:
dyke = OrganizationInstrument(name = 'Dyke', cost = 8, completion_time = 5, protection_level = 0.7, status = 1)
wetland = OrganizationInstrument(name = 'Wetland', cost = 5,  completion_time = 2, protection_level = 0.4, status = 1)  
options_list = [dyke, wetland]

## Batchrunner

In [4]:
params = {
    "seed": 42,
    "options_list": None,
    "number_of_households": 100,
    "flood_map_choice": 'harvey',
    "network": 'watts_strogatz', #["erdos_renyi", "barabasi_albert", "watts_strogatz"],
    "probability_of_network_connection": 0.4,
    "number_of_edges" : 3,"number_of_nearest_neighbours" : 5,

    "flood_probability" : 0.05,
    "economic_status": 'neutral',
    "intention_action_gap" : 0.3,
    "low_threshold" : 0.6,
    "medium_threshold" : 0.7,
    "high_threshold" : 0.8,
    "upper_budget_threshold" : 7000,
    "lower_budget_threshold" : 3000,
    "elevation_time" : 4,
    "elevation_cost" : 5000,
    "elevation_protection" : 0.3,
    "elevation_effectiveness" : 1,
    "wet_proofing_time" : 2,
    "wet_proofing_cost" : 3000,
    "wet_proofing_effectiveness" : 0.4,
    "dry_proofing_time" : 1,
    "dry_proofing_cost" : 1500,
    "dry_proofing_protection" : 1,
    "dry_proofing_effectiveness" : 0.85,
    "max_damage_costs" : 5000,
    "flood_risk_threshold": 1.5,
    "public_concern_threshold": 0.6,
    "damage_threshold": 0.3,
    "high_risk_bound": 2.9,
    "lower_risk_bound": 1.9,
    "gov_detector": 0,
    "gov_structure": 'centralised',
          }

results = batch_run(
    AdaptationModel,
    parameters=params,
    iterations=5,
    max_steps=120,
    number_processes=1,
    data_collection_period=1,
    display_progress=True,
)

  0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
df_basecase_no_opt = pd.DataFrame(results)
df_basecase_no_opt

,RunId,iteration,Step,seed,options_list,number_of_households,flood_map_choice,network,probability_of_network_connection,number_of_edges,...,Average Adaptation Motivation,Average External Influence,Flood,AgentID,FloodDamageActual,IsAdapted,Adaptation_Motivation,Financial_Loss,Agenda,Decision
0,0,0,0,42,None,100,harvey,watts_strogatz,0.4,3,...,0.493328,0.499814,False,1,0.000000,False,0.376995,0.000000,None,None
1,0,0,0,42,None,100,harvey,watts_strogatz,0.4,3,...,0.493328,0.499814,False,2,0.000000,False,0.309923,0.000000,None,None
2,0,0,0,42,None,100,harvey,watts_strogatz,0.4,3,...,0.493328,0.499814,False,3,0.000000,False,0.413100,0.000000,None,None
3,0,0,0,42,None,100,harvey,watts_strogatz,0.4,3,...,0.493328,0.499814,False,4,0.000000,False,0.590549,0.000000,None,None
4,0,0,0,42,None,100,harvey,watts_strogatz,0.4,3,...,0.493328,0.499814,False,5,0.000000,False,0.447817,0.000000,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61100,4,4,120,42,None,100,harvey,watts_strogatz,0.4,3,...,0.647990,0.287535,True,97,0.000000,False,0.326966,0.000000,None,None
61101,4,4,120,42,None,100,harvey,watts_strogatz,0.4,3,...,0.647990,0.287535,True,98,0.000000,False,1.000000,0.000000,None,None
61102,4,4,120,42,None,100,harvey,watts_strogatz,0.4,3,...,0.647990,0.287535,True,99,0.048208,False,1.000000,1120.416173,None,None
61103,4,4,120,42,None,100,harvey,watts_strogatz,0.4,3,...,0.647990,0.287535,True,100,0.000000,False,0.329513,0.000000,None,None


In [6]:
# g = sns.FacetGrid(df_basecase_no_opt, col_wrap = 5, col = 'Iteratie', )
# g.fig.suptitle("Hoeveelheid plastic afvalproducten voor Basecase in verschillende scenario's, verschillende iteraties")
# g.map(sns.lineplot,x = df_basecase_no_opt['Step'], y= df_basecase_waste['basecase0'], color = 'green', label = 'basecase0')
# g.map(sns.lineplot, x = df_basecase_waste['Stap'], y= df_basecase_waste['basecase1'], color = 'red', label = 'basecase1')
# g.map(sns.lineplot, x = df_basecase_waste['Stap'], y= df_basecase_waste['basecase2'], color = 'blue', label = 'basecase2')
# g.map(sns.lineplot,x = df_basecase_waste['Stap'], y= df_basecase_waste['basecase3'], color = 'pink', label = 'basecase3' )
# plt.legend()
# g.set_axis_labels(x_var = "Tijd [# maanden]", y_var = "Hoeveelheid plastic afvalproduct [#] *10^6")
# g.set(xticks = [6, 12, 18, 24, 30, 36, 42, 48, 54, 60])
# plt.savefig("WASTE_PLASTIC_BASECASE.png", dpi=100,bbox_inches='tight')


# plt.show()